In [30]:
import numpy as np
import pandas as pd
import cvxpy as cp
from sklearn.model_selection import train_test_split
from copy import deepcopy

In [31]:
train1 = pd.read_csv(r'/Users/niharika/Documents/Niharika/Third Quarter/ECS 271 ML/Assignment3/studentspen-train.csv', header=0)
test = pd.read_csv(r'/Users/niharika/Documents/Niharika/Third Quarter/ECS 271 ML/Assignment3/studentsdigits-test.csv', header=0)
x_test = np.array(test)

In [32]:
# train1, cv = train_test_split(train2, test_size = 0.2)

In [33]:
poly_maps = []
factorial = np.zeros(20)
def comp_factorial():
    global factorial
    factorial[0] = 1
    for i in range(1, factorial.shape[0]):
        factorial[i] = i * factorial[i - 1]

comp_factorial()

def get_poly_maps(c_map, index, degree):
    if (index == c_map.shape[0]):
        if (np.sum(c_map) == degree):
            poly_maps.append(deepcopy(c_map))
        return
    for i in range(degree + 1):
        c_map[index] = i
        get_poly_maps(c_map, index + 1, degree)
    c_map[index] = 0

def poly_kernel(x, degree):
    global poly_maps
    poly_maps = []
    D = x.shape[1]

    c_map = np.zeros(D)
    get_poly_maps(c_map, 0, degree)
    poly_maps = np.array(poly_maps)
    xk = np.zeros((x.shape[0], poly_maps.shape[0]))
    
    for i in range(xk.shape[0]):
        for j in range(xk.shape[1]):
            c = factorial[degree]
            xk[i][j] = 1.0
            for k in range(len(poly_maps[j])):
                c /= factorial[int(poly_maps[j][k])]
                xk[i][j] *= x[i][k] ** poly_maps[j][k]
            c = c ** 0.5
            xk[i][j] *= c

    return xk

In [34]:
w_val = []
b_val = []

for cls in range(0,10):
    train = train1.copy(deep=True)
    train.loc[train.Digit != cls, 'Digit'] = -1
    train.loc[train.Digit == cls, 'Digit'] = 1
    x_train = train[['x3','y3','x4','y4','x5','y5','x6','y6']]
    x = np.array(x_train)
    y_train = train[['Digit']]
    y = np.array(y_train)
    xk = poly_kernel(x, 2)
    D = xk.shape[1]
    w = cp.Variable((D, 1))
    b = cp.Variable()
    C = 2
    psi = cp.Variable((xk.shape[0], 1))
    objective = cp.Minimize(cp.sum(cp.square(w)) * 0.5 + C*cp.sum(cp.square(psi)))
    constraints = [cp.multiply(y, (xk @ w + b)) + psi >= 1, psi >= 0]
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.ECOS)
    w_val.append(w.value)
    b_val.append(b.value)

In [35]:
# x_test = np.array(cv[['x3','y3','x4','y4','x5','y5','x6','y6']])
# y_test = np.array(cv[['Digit']])

In [36]:
xk_test = poly_kernel(x_test, 2)
total = 0
correct = 0
file = open(r'/Users/niharika/Documents/pred_kernel.txt','w')
for i in range(len(x_test)):
#     total += 1
    pred = []
    for j in range(len(w_val)):
        temp = np.dot(np.transpose(w_val[j]),xk_test[i]) + b_val[j]
        pred.append(temp)
    file.write(str(pred.index(max(pred))))
    file.write('\n')
#     if pred.index(max(pred)) == y_test[i]:
#         correct += 1
file.close()

# print(total,correct)